In [45]:
import systemml as sml
from pyspark import SparkContext
from systemml import MLContext, dml

In [46]:
sc=SparkContext.getOrCreate()
ml=MLContext(sc)
print(ml.info())

Archiver-Version: Plexus Archiver
Artifact-Id: systemml
Build-Jdk: 1.8.0_151
Build-Time: 2018-08-17 05:58:31 UTC
Built-By: reinwald
Created-By: Apache Maven 3.3.9
Group-Id: org.apache.systemml
Main-Class: org.apache.sysml.api.DMLScript
Manifest-Version: 1.0
Minimum-Recommended-Spark-Version: 2.1.0
Version: 1.2.0



In [67]:
import numpy as np
from sklearn import datasets
#import systemml as sml
# Load the diabetes dataset
diabetes = datasets.load_diabetes()
# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]
# Split the data into training/testing sets
X_train = diabetes_X[:-20]
X_test = diabetes_X[-20:]
# Split the targets into training/testing sets
y_train = diabetes.target[:-20]
y_test = diabetes.target[-20:]

In [70]:
  script= """
         ones=matrix(1, rows=nrow(X), cols=1)
         X=cbind(X,ones)
         max_iter=100
         w=matrix(0, rows=ncol(X), cols=1)
         for (i in 1:max_iter){
         XtX=t(x) %*% X
         dw= XtX %*% w - t(X) %*% y
         alpha=(t(dw) %** dw)/(t(dw) %*% XtX %*% dw)
         w=w-dw * alpha
         }
         bias=as.scalar(w[nrow(w),1])
         w=w[1:nrow(w) -1]
   """

In [71]:
prog=dml(script).input(X=X_train, y=y_train).output('w').output('bias')
w, bias=ml.execute(prog).get('w', 'bias')
w=w.toNumpy()

TypeError: Expected 2-dimensional ndarray, instead passed 1-dimensional ndarray

In [73]:
from systemml import dmlFromResource
prog=dmlFromResource('scripts/algorithms/LinearRegDS.dml').input(X=X_train,y=y_train).input('$icpt', 1.0).output('beta_out')
w=ml.execute(prog).get('beta_out')
w=w.toNumpy()
bias=w[1]

TypeError: Expected 2-dimensional ndarray, instead passed 1-dimensional ndarray